<a href="https://colab.research.google.com/github/NSambhajiS/NLP-Labs/blob/main/Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import math

In [3]:
# 1. Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]


In [4]:
# 2. Multi-Head Attention Mechanism
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = d_model // num_heads

        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)
        self.wo = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = torch.matmul(Q, K.transpose(-2, -1))
        d_k = K.shape[-1]
        scaled_attention_logits = matmul_qk / math.sqrt(d_k)
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        return torch.matmul(attention_weights, V), attention_weights

    def forward(self, x):
        batch_size = x.shape[0]

        Q = self.wq(x).view(batch_size, -1, self.num_heads, self.depth).transpose(1,2)
        K = self.wk(x).view(batch_size, -1, self.num_heads, self.depth).transpose(1,2)
        V = self.wv(x).view(batch_size, -1, self.num_heads, self.depth).transpose(1,2)

        scaled_attention, _ = self.scaled_dot_product_attention(Q, K, V)

        concat_attention = scaled_attention.transpose(1,2).contiguous().view(batch_size, -1, self.d_model)

        return self.wo(concat_attention)


In [5]:
# 3. Feed Forward Network
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForwardNetwork, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))


In [6]:
# 4. Transformer Encoder Layer
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForwardNetwork(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output = self.mha(x)
        x = self.norm1(x + self.dropout(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x


In [7]:
# 5. Transformer Model
class Transformer(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, d_ff, num_layers):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(input_dim, d_model)
        self.pos_encoding = PositionalEncoding(d_model)
        self.encoder_layers = nn.ModuleList([TransformerEncoderLayer(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.fc_out = nn.Linear(d_model, input_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.encoder_layers:
            x = layer(x)
        return self.fc_out(x)


In [8]:
# Example usage
input_dim = 1000  # Vocabulary size
d_model = 128
num_heads = 8
d_ff = 512
num_layers = 6

model = Transformer(input_dim, d_model, num_heads, d_ff, num_layers)
example_input = torch.randint(0, 1000, (32, 10))  # Batch size 32, sequence length 10
output = model(example_input)
print(output.shape)  # Expected: [32, 10, 1000]


torch.Size([32, 10, 1000])
